In [1]:
# 初始化环境，可以设置成全局的环境变量，也可以在代码中指定
from openai import OpenAI
import os


# 填写你自己的key
os.environ['OPENAI_API_KEY'] = '${OPENAI_API_KEY}'

client = OpenAI()

In [2]:
# 1.  创建助手
assistant = client.beta.assistants.create(
    # 助手名称
    name="Math Tutor",
    # 填写提示词
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    # 助手的描述
    description="数学老师",
    tools=[
        # 开启代码解释器
        {"type": "code_interpreter"}
    ],
    # 模型
    model="gpt-3.5-turbo-1106"
)
print(assistant.model_dump_json())

{"id":"asst_wpqLs7IRt9Wg41rk5Mo1JEK2","created_at":1713432006,"description":"数学老师","instructions":"You are a personal math tutor. Write and run code to answer math questions.","metadata":{},"model":"gpt-3.5-turbo-1106","name":"Math Tutor","object":"assistant","tools":[{"type":"code_interpreter"}],"response_format":"auto","temperature":1.0,"tool_resources":{"code_interpreter":{"file_ids":[]},"file_search":null},"top_p":1.0}


In [ ]:
# 获取assistants列表
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
print(my_assistants.model_dump_json())

In [ ]:
# 获取assistants详情
my_assistant = client.beta.assistants.retrieve(assistant.id)
print(my_assistant.model_dump_json())

In [ ]:
# 删除助手
response = client.beta.assistants.delete("asst_0sxLXccq88BZXwTpes2xlm8J")
print(response.model_dump_json())

In [3]:
# 2. 创建线程
empty_thread = client.beta.threads.create()
print(empty_thread.model_dump_json())

{"id":"thread_MwEzGiETgjoSkIuFTqLTjLC1","created_at":1713432809,"metadata":{},"object":"thread","tool_resources":{"code_interpreter":null,"file_search":null}}


In [5]:
# 获取线程
my_thread = client.beta.threads.retrieve(empty_thread.id)
print(my_thread.model_dump_json())

{"id":"thread_MwEzGiETgjoSkIuFTqLTjLC1","created_at":1713432809,"metadata":{},"object":"thread","tool_resources":{"code_interpreter":{"file_ids":[]},"file_search":null}}


In [ ]:
# 删除线程
response = client.beta.threads.delete("thread_uztos9m97ckTgHmzuqpNFjXe")
print(response.model_dump_json())

In [6]:
# 3. 向主题添加消息
message = client.beta.threads.messages.create(
    thread_id=my_thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)
print(message.model_dump_json())

{"id":"msg_hqEy9v5ucrXb31IY7UllD0TZ","assistant_id":null,"attachments":[],"completed_at":null,"content":[{"text":{"annotations":[],"value":"I need to solve the equation `3x + 11 = 14`. Can you help me?"},"type":"text"}],"created_at":1713432856,"incomplete_at":null,"incomplete_details":null,"metadata":{},"object":"thread.message","role":"user","run_id":null,"status":null,"thread_id":"thread_MwEzGiETgjoSkIuFTqLTjLC1"}


In [ ]:
# 获取线程下的所有消息
thread_messages = client.beta.threads.messages.list(thread_id=my_thread.id)
print(thread_messages.model_dump_json())

In [ ]:
# 获取消息详情
message = client.beta.threads.messages.retrieve(
    message_id=message.id,
    thread_id=message.thread_id,
)
print(message.model_dump_json())

In [7]:
# 4. 运行助手 
run = client.beta.threads.runs.create(
    thread_id=my_thread.id,
    assistant_id=assistant.id
)
print(run.model_dump_json())

{"id":"run_aE3RTVjfo0B6Mixzh1rK9MB7","assistant_id":"asst_wpqLs7IRt9Wg41rk5Mo1JEK2","cancelled_at":null,"completed_at":null,"created_at":1713432874,"expires_at":1713433474,"failed_at":null,"incomplete_details":null,"instructions":"You are a personal math tutor. Write and run code to answer math questions.","last_error":null,"max_completion_tokens":null,"max_prompt_tokens":null,"metadata":{},"model":"gpt-3.5-turbo-1106","object":"thread.run","required_action":null,"response_format":"auto","started_at":null,"status":"queued","thread_id":"thread_MwEzGiETgjoSkIuFTqLTjLC1","tool_choice":"auto","tools":[{"type":"code_interpreter"}],"truncation_strategy":{"type":"auto","last_messages":null},"usage":null,"temperature":1.0,"top_p":1.0,"tool_resources":{}}


In [ ]:
# 返回属于线程的运行列表。
runs = client.beta.threads.runs.list(run.thread_id)
print(runs.model_dump_json())

In [ ]:
# 列出运行详细步骤
run_steps = client.beta.threads.runs.steps.list(
    thread_id=my_thread.id,
    run_id=run.id
)
print(run_steps.model_dump_json())

In [8]:
# 5. 检查运行状态
run = client.beta.threads.runs.retrieve(
    thread_id=my_thread.id,
    run_id=run.id
)
print(run.model_dump_json())

{"id":"run_aE3RTVjfo0B6Mixzh1rK9MB7","assistant_id":"asst_wpqLs7IRt9Wg41rk5Mo1JEK2","cancelled_at":null,"completed_at":1713432881,"created_at":1713432874,"expires_at":null,"failed_at":null,"incomplete_details":null,"instructions":"You are a personal math tutor. Write and run code to answer math questions.","last_error":null,"max_completion_tokens":null,"max_prompt_tokens":null,"metadata":{},"model":"gpt-3.5-turbo-1106","object":"thread.run","required_action":null,"response_format":"auto","started_at":1713432874,"status":"completed","thread_id":"thread_MwEzGiETgjoSkIuFTqLTjLC1","tool_choice":"auto","tools":[{"type":"code_interpreter"}],"truncation_strategy":{"type":"auto","last_messages":null},"usage":{"completion_tokens":88,"prompt_tokens":348,"total_tokens":436},"temperature":1.0,"top_p":1.0,"tool_resources":{}}


In [9]:
# 6. 显示最终响应
thread_messages = client.beta.threads.messages.list(thread_id=my_thread.id)
print(thread_messages.model_dump_json())

{"data":[{"id":"msg_ZyZKpwD9YkFTcbWIOwMkSM4y","assistant_id":"asst_wpqLs7IRt9Wg41rk5Mo1JEK2","attachments":[],"completed_at":null,"content":[{"text":{"annotations":[],"value":"The solution to the equation \\(3x + 11 = 14\\) is \\(x = 1\\)."},"type":"text"}],"created_at":1713432880,"incomplete_at":null,"incomplete_details":null,"metadata":{},"object":"thread.message","role":"assistant","run_id":"run_aE3RTVjfo0B6Mixzh1rK9MB7","status":null,"thread_id":"thread_MwEzGiETgjoSkIuFTqLTjLC1"},{"id":"msg_hqEy9v5ucrXb31IY7UllD0TZ","assistant_id":null,"attachments":[],"completed_at":null,"content":[{"text":{"annotations":[],"value":"I need to solve the equation `3x + 11 = 14`. Can you help me?"},"type":"text"}],"created_at":1713432856,"incomplete_at":null,"incomplete_details":null,"metadata":{},"object":"thread.message","role":"user","run_id":null,"status":null,"thread_id":"thread_MwEzGiETgjoSkIuFTqLTjLC1"}],"object":"list","first_id":"msg_ZyZKpwD9YkFTcbWIOwMkSM4y","last_id":"msg_hqEy9v5ucrXb31IY7

In [10]:
## 再来一遍
# 3. 向主题添加消息
message = client.beta.threads.messages.create(
    thread_id=my_thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 17`. Can you help me?"
)
print(message.model_dump_json())

{"id":"msg_UEsHtU68SnKgQGKGe28aJU2e","assistant_id":null,"attachments":[],"completed_at":null,"content":[{"text":{"annotations":[],"value":"I need to solve the equation `3x + 11 = 17`. Can you help me?"},"type":"text"}],"created_at":1713432919,"incomplete_at":null,"incomplete_details":null,"metadata":{},"object":"thread.message","role":"user","run_id":null,"status":null,"thread_id":"thread_MwEzGiETgjoSkIuFTqLTjLC1"}


In [ ]:
# 获取线程下的所有消息
thread_messages = client.beta.threads.messages.list(thread_id=my_thread.id)
print(thread_messages.model_dump_json())

In [11]:
# 4. 运行助手 
run = client.beta.threads.runs.create(
    thread_id=my_thread.id,
    assistant_id=assistant.id
)
print(run.model_dump_json())

{"id":"run_shkoZzYF4ricCqE1HRMwFrPA","assistant_id":"asst_wpqLs7IRt9Wg41rk5Mo1JEK2","cancelled_at":null,"completed_at":null,"created_at":1713432923,"expires_at":1713433523,"failed_at":null,"incomplete_details":null,"instructions":"You are a personal math tutor. Write and run code to answer math questions.","last_error":null,"max_completion_tokens":null,"max_prompt_tokens":null,"metadata":{},"model":"gpt-3.5-turbo-1106","object":"thread.run","required_action":null,"response_format":"auto","started_at":null,"status":"queued","thread_id":"thread_MwEzGiETgjoSkIuFTqLTjLC1","tool_choice":"auto","tools":[{"type":"code_interpreter"}],"truncation_strategy":{"type":"auto","last_messages":null},"usage":null,"temperature":1.0,"top_p":1.0,"tool_resources":{}}


In [12]:
thread_messages = client.beta.threads.messages.list(thread_id=my_thread.id)
print(thread_messages.model_dump_json())

{"data":[{"id":"msg_WIfvc4vGm9rGnnrODW8uWrIi","assistant_id":"asst_wpqLs7IRt9Wg41rk5Mo1JEK2","attachments":[],"completed_at":null,"content":[{"text":{"annotations":[],"value":"The solution to the equation \\(3x + 11 = 17\\) is \\(x = 2\\)."},"type":"text"}],"created_at":1713432927,"incomplete_at":null,"incomplete_details":null,"metadata":{},"object":"thread.message","role":"assistant","run_id":"run_shkoZzYF4ricCqE1HRMwFrPA","status":null,"thread_id":"thread_MwEzGiETgjoSkIuFTqLTjLC1"},{"id":"msg_UEsHtU68SnKgQGKGe28aJU2e","assistant_id":null,"attachments":[],"completed_at":null,"content":[{"text":{"annotations":[],"value":"I need to solve the equation `3x + 11 = 17`. Can you help me?"},"type":"text"}],"created_at":1713432919,"incomplete_at":null,"incomplete_details":null,"metadata":{},"object":"thread.message","role":"user","run_id":null,"status":null,"thread_id":"thread_MwEzGiETgjoSkIuFTqLTjLC1"},{"id":"msg_ZyZKpwD9YkFTcbWIOwMkSM4y","assistant_id":"asst_wpqLs7IRt9Wg41rk5Mo1JEK2","attach